# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

OPENAI_API_KEY=****lj8A
GROQ_API_KEY=****J18N
LANGSMITH_API_KEY=****c376
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio
import asyncio
import sys

# Windows-specific fix for asyncio subprocesses (MCP needs this)
if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

UnsupportedOperation: fileno

Create an agent with the MCP-provided time tools.

In [ ]:
from langchain.agents import create_agent
from langchain_groq import ChatGroq

# Initialize the Groq model
# Using llama-3.1-8b-instant as it supports tool calling natively
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2
)

agent_with_mcp = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [ ]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()